In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

from matplotlib import pyplot as plt
from dotenv import load_dotenv

from scripts.merge import merge_cols, complete_df
from scripts.convert import convert_to_holy_metric, find_unit

load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = os.getenv('DPI')

if pc_dpi is None:
    pc_dpi = 100
if pc_dpi is not None:
    pc_dpi = int(pc_dpi)


# <u>Pre Cleaning : </u>

<u>Objectifs :</n>

1. Formatter les données
2. Identifier et retenir les variables indispensables à l'analyse et la construction de modeles predictifs
3. Convertir les données au format metrique/internationnal (base 10)
4. Exporter un dataset pret pour une analyse exploratoire

## <u> 1. Formattage des données </u>

In [ ]:
# Files

twenty_fifteen_bm = "data/2015-building-energy-benchmarking.csv"
twenty_sixteen_bm = "data/2016-building-energy-benchmarking.csv"


In [ ]:
df_fifteen = pd.read_csv(twenty_fifteen_bm, dtype="unicode", low_memory=False)
df_sixteen = pd.read_csv(twenty_sixteen_bm, dtype="unicode", low_memory=False)


In [ ]:
print("info : ", df_fifteen.info())

In [ ]:
print("info : ", df_sixteen.info())


In [ ]:
id_fifteen_list = list(df_fifteen["OSEBuildingID"])
id_sixteen_list = list(df_sixteen["OSEBuildingID"])

only_in_fifteen = [ident for ident in id_fifteen_list if ident not in id_sixteen_list]
only_in_sixteen = [ident for ident in id_sixteen_list if ident not in id_fifteen_list]

print(
    f"""{len(only_in_fifteen)} batiments ne sont présents que dans le benchmark 2015,\
 et {len(only_in_sixteen)} batiments ne sont présents que dans le benchmark 2016 """
  )


In [ ]:
col_onl_fifteen = [col for col in df_fifteen.columns if col not in df_sixteen.columns]

col_onl_sixteen = [col for col in df_sixteen.columns if col not in df_fifteen.columns]

print("Nombre de variables non communes aux deux datasets : ", len(col_onl_fifteen) + len(col_onl_sixteen))


In [ ]:
print(col_onl_sixteen)

In [ ]:
print(col_onl_fifteen)

In [ ]:
df_fifteen["Location"].describe().top


### Observations :
- Les deux datasets mentionnent la presence d'outliers, mais les metadonnees n'informent pas ce qui les determinent. Suppression des outliers.
- Les données de localisation de l'annee 2015 sont presentes au format d'un dictionnaire, format plus efficace que les nombreuses variables de 2016. On pourra compresser les donnees de 2016 pour obtenir ce format
- Certaines colonnes ne sont presentes que dans le dataset 2015, ces variables seront supprimées (droplist), d'autres ont un nom different et seront renommées
- Certaines colonnes sont inutiles a l'etude et a la construction d'un modele mais apportent des informations interessantes sur les batiments si le besoin est. --> Creation d'un dataset Meta
- L'etude specifie l'exclusion des batiments residentiels : identification et suppression
- Plusieurs batiments ne sont pas communs aux deux datasets, on utilsera par defaut les informations les plus recentes (2016), et on importera les données manquantes

In [ ]:
# Outliers :

df_sixteen = df_sixteen[df_sixteen["Outlier"].isna()]
df_fifteen = df_fifteen[df_fifteen["Outlier"].isna()]


In [ ]:
rename_sixteen = {
    "Comments": "Comment",
    "TotalGHGEmissions": "GHGEmissions(MetricTonsCO2e)",
    "GHGEmissionsIntensity": "GHGEmissionsIntensity(kgCO2e/ft2)",
    }

droplist = [
    "Zip Codes", "Zipcode", "SPD Beats", "2010 Census Tracts", "City Council Districts",
    "Seattle Police Department Micro Community Policing Plan Areas", "OtherFuelUse(kBtu)",  # OtherFuelUse not present in both datasets
    "SPD Beats", "CouncilDistrictCode", "Outlier", "DataYear", "ZipCode", "NaturalGas(therms)",  # Natural gas already present in another unit
    ]

# Columns not relevant to study but informative on the properties
meta_df_cols = [
    "TaxParcelIdentificationNumber", "YearsENERGYSTARCertified", "Comment", "Location", "OSEBuildingID",
    "PrimaryPropertyType", "PropertyName", "YearBuilt"
    ]

meta_keep = ["OSEBuildingID", "PrimaryPropertyType", "PropertyName", "YearBuilt"]

compress_cols = ["Address", "City", "State", "Latitude", "Longitude"]


In [ ]:
df_fifteen = df_fifteen.drop(columns=droplist, errors="ignore")
df_sixteen = df_sixteen.drop(columns=droplist, errors="ignore")


In [ ]:
# Using 2015 synthax, compression of compress cols & placing data in new col

df_sixteen["Location"] = np.nan

merge_cols(origin_col_list=compress_cols, target_col_name="Location", dataframe=df_sixteen)

df_sixteen.drop(columns=compress_cols, inplace=True)


In [ ]:
# renaming cols for merger

df_sixteen.rename(columns=rename_sixteen, inplace=True)

for col in df_sixteen.columns:
    if col not in df_fifteen.columns:
        raise BaseException("Mismatch")


In [ ]:
# merging : 
to_merge = df_fifteen[df_fifteen["OSEBuildingID"].isin(only_in_fifteen)]

df_seattle = pd.concat([df_sixteen, to_merge])

if len(df_seattle[df_seattle.duplicated()]) > 0:
    print("Duplicates, edit code ...")
else:
    print("No duplicate, merger ok")


In [ ]:
# Created 2 meta dataframes and dropped the cols in main dfs

meta_df_fifteen = df_fifteen[meta_df_cols]
meta_df_sixteen = df_sixteen[meta_df_cols]

meta_drop = [col for col in meta_df_cols if col not in meta_keep]

df_fifteen.drop(columns=meta_drop, inplace=True)
df_sixteen.drop(columns=meta_drop, inplace=True)
df_seattle.drop(columns=meta_drop, inplace=True)


In [ ]:
btypes_fifteen = list(df_fifteen["BuildingType"].unique())
btypes_sixteen = list(df_sixteen["BuildingType"].unique())

btypes_both = btypes_fifteen
[btypes_both.append(btype) for btype in btypes_sixteen if btype not in btypes_both]

print(btypes_both)

In [ ]:
# Residential startswith multifamily

residential = [btype for btype in btypes_both if str(btype).lower().startswith("multifamily")]
print(residential)


In [ ]:
# Dropping residential

df_seattle = df_seattle[~df_seattle["BuildingType"].isin(residential)]

# Check : 

btype_after_pass = list(df_seattle["BuildingType"].unique())

for btype in btype_after_pass:
    if btype in residential:
        print("Suppression Failed, advise")
        break
    
print("Suppression OK")



In [ ]:
# Convert

unit_stop_chars = ["(", ")", "GFA"]

manual_ignore = ["Electricity(kWh)", "GHGEmissions(MetricTonsCO2e)", "GHGEmissionsIntensity(kgCO2e/ft2)"]

unit_cols = [col for col in df_seattle.columns if (any(char in col for char in unit_stop_chars) and col not in manual_ignore)]


In [ ]:
unit_dict = dict.fromkeys(unit_cols)

for col in unit_dict.keys(): 
    unit = find_unit(var_name=col, convert=True)["converted_unit"]

    if "GFA" in col:
        new_name = col.replace("GFA", "Area(SquareMetre)")
    elif ("("and ")") in col:
        start, end = col.find("("), col.find(")")
        before = col[:start + 1]
        after = col[end:]
        new_name = f"{before}{unit}{after}"
    
    df_seattle[new_name] = np.nan
    unit_dict[col] = new_name

for key, value in unit_dict.items():

    for index, series in df_seattle.iterrows():
        df_seattle.at[index, value] = convert_to_holy_metric(data=series[key], var_name=key)

df_seattle.drop(columns=unit_dict.keys(), inplace=True)


In [ ]:
df_seattle.columns


In [ ]:
# EPA has determined that source energy is the most equitable [...] cf. sources
# |^| removing cols starting with site, except SiteEnergyUse(kWh) (no source)

ignore_site = ["SiteEnergyUse(kWh)", "SiteEnergyUseWN(kWh)"]

site_cols = [col for col in df_seattle.columns if (col.startswith("Site") and col not in ignore_site)]
df_seattle.drop(columns=site_cols, inplace=True)


In [ ]:
# Steam and Natural gas might be uncommon - Plotting ...

fig, (ax1) = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(22, 10),
    dpi=pc_dpi,
)

ax1 = sns.boxplot(
    data=df_seattle[["SteamUse(kWh)", "NaturalGas(kWh)", "SiteEnergyUse(kWh)"]]
)


###
# Titles/Lables

ax1.set_ylim(0, 0.5 * 1e7)
fig.suptitle("Representation de l'utilisation de la vapeur et du gaz naturel au sein du dataset\
 ; visualistion de l'energie totale pour reference")
#
###
fig.tight_layout()
plt.show()


In [ ]:
df_seattle.drop(columns=["SteamUse(kWh)"], inplace=True)


In [ ]:
df_seattle.columns

In [ ]:
manual_drop = [
    "ListOfAllPropertyUseTypes", "SecondLargestPropertyUseType",
    "ThirdLargestPropertyUseType", "DefaultData", "ComplianceStatus",
    "SecondLargestPropertyUseTypeArea(SquareMetre)", "ThirdLargestPropertyUseTypeArea(SquareMetre)",
    ]

df_seattle.drop(columns=manual_drop, inplace=True)
